<a href="https://colab.research.google.com/github/amelierueeck/ULM-25-authorship-profiling/blob/main/src/BERT_probing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tuned model:

`KonradBRG/bert-lora-for-author-profiling`

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
import sys
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device('cpu')

In [ ]:
#get data (put files into colab folder)

# load splits
train_df = pd.read_csv("data_train.csv")
val_df   = pd.read_csv("data_val.csv")
test_df  = pd.read_csv("data_test.csv")

print(train_df.head())

                                                text  age  gender
0  Well...kind of an interesting little thingy I ...    0    male
1  Lonliness dominates you. You can hide it well,...    0    male
2  Wearing:New jacket, hoody, dickies, vans, and ...    0  female
3  Well I got into another verbal sparring match ...    3    male
4  Hey everyone.&nbsp; I am in Brugge, Belgium.&n...    2    male


In [ ]:
print(f"Train instances: {train_df.shape[0]}")
print(f"Val instances: {val_df.shape[0]}")
print(f"Test instances: {test_df.shape[0]}")

Train instances: 620813
Val instances: 68980
Test instances: 37919


In [ ]:
#load pretrained BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True).to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
#helper function
def get_layer(split, idx):
  """Get the layer at idx in the given split (test/val/train)"""
  data = np.load(f"{split}_layers.npz")
  return data[f"layer{idx}"]

In [ ]:
#define probing function
def train_probe(X_train, y_train, X_val, y_val):
    clf = LogisticRegression(max_iter=5000, class_weight="balanced", multi_class="auto")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    return accuracy_score(y_val, y_pred), clf

In [ ]:
#gender:
y_train_gender = train_df["gender"].values
y_val_gender   = val_df["gender"].values
y_test_gender  = test_df["gender"].values

In [ ]:
#age:
y_train_age = train_df["age"].values
y_val_age   = val_df["age"].values
y_test_age  = test_df["age"].values

In [ ]:
#run probes for gender:

results_gender = []
for layer_idx in range(13):
    acc, _ = train_probe(
        get_layer("train", layer_idx), y_train_gender,
        get_layer("test", layer_idx), y_test_gender
    )
    results_gender.append(acc)
    print(f"[Gender] Layer {layer_idx}: probe accuracy = {acc:.3f}")

In [ ]:
results_age = []
for layer_idx in range(13):
    acc, _ = train_probe(
        get_layer("train", layer_idx), y_train_age,
        get_layer("test", layer_idx), y_test_age
    )
    results_age.append(acc)
    print(f"[Age] Layer {layer_idx}: probe accuracy = {acc:.3f}")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(13), results_gender, marker="o", label="Gender")
plt.plot(range(13), results_age, marker="s", label="Age")
plt.xlabel("Layer")
plt.ylabel("Probe Accuracy")
plt.title("Probing Pretrained BERT for Gender vs Age")
plt.legend()
plt.show()